#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt", encoding="utf-8")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [2]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [3]:
embeddings=OllamaEmbeddings(model="llama2")
db=FAISS.from_documents(docs,embeddings)
db

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2032\331990801.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings(model="llama2")


In [4]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [6]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [7]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='a4cce3ad-c407-48d2-8cd6-c07836c8c138', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  15690.737),
 (Document(id='32ae9e07-761a-42c5-ae1a-bf658783b26e', metadata={'source': 'speech.txt'}, p

In [8]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[1.851627230644226,
 -3.073892593383789,
 1.9968559741973877,
 1.4900089502334595,
 -0.9077807664871216,
 0.6273611187934875,
 1.5782333612442017,
 -0.5514776110649109,
 0.9163244366645813,
 -2.011291980743408,
 1.3220398426055908,
 -1.9161951541900635,
 -0.6701374650001526,
 1.5195260047912598,
 -0.5097979307174683,
 -2.026827573776245,
 -0.41212570667266846,
 0.04242005944252014,
 0.7072376608848572,
 -1.8591690063476562,
 -0.6834448575973511,
 -1.042574167251587,
 2.160907506942749,
 -1.971352219581604,
 0.748491108417511,
 -1.0021579265594482,
 0.37059667706489563,
 -1.6514081954956055,
 0.2133689671754837,
 -0.8416838049888611,
 2.055795431137085,
 -2.7911624908447266,
 -2.6446573734283447,
 3.7815914154052734,
 2.3162429332733154,
 -5.087841033935547,
 -1.0436333417892456,
 1.4130918979644775,
 -1.2553565502166748,
 -1.3667354583740234,
 -0.4926852881908417,
 -2.004255533218384,
 2.1183786392211914,
 0.3405381143093109,
 0.9395475387573242,
 0.5849294662475586,
 -0.25697293877601

In [9]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='a4cce3ad-c407-48d2-8cd6-c07836c8c138', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='32ae9e07-761a-42c5-ae1a-bf658783b26e', metadata={'source': 'speech.txt'}, page_content='To 

In [11]:
### Saving And Loading
db.save_local("faiss_index")

In [16]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [17]:
docs

[Document(page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.', metadata={'source': 'speech.txt'}),
 Document(page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the prid